In [1]:
import matplotlib.pyplot as plt 
from matplotlib.pyplot import cm
import numpy as np
import pickle

%matplotlib

Using matplotlib backend: TkAgg


# View times in a dataframe

In [2]:
import pandas as pd

# unpickle times and states
filename = "potential_pair.pkl"
with open("data/" + filename, 'rb') as handle:
    times_and_states = pickle.load(handle)

# create dataframe (columns are labelled by number of coupled DFNs)
# TODO: unpack list of solve times
df = pd.DataFrame(times_and_states)
df

,1,3,4,8,16,32,64,128,256,512
states,642,1930,2573,5145,10289,20577,41153,82305,164609,329217
setup times,"[0.2611990440000227, 7.3940000220318325e-06, 6...","[0.4570253970000522, 8.291999847642728e-06, 5....","[0.59252695400005, 5.467999926622724e-06, 4.93...","[0.9856216999999106, 5.633999990095617e-06, 5....","[1.884182133999957, 5.196999836698524e-06, 6.9...","[3.2813403179998204, 6.159000122352154e-06, 6....","[8.394405901000027, 5.8829998579312814e-06, 6....","[23.740155207000043, 6.300999984887312e-06, 5....","[81.0953574969999, 5.597000381385442e-06, 5.30...","[266.4875437810001, 5.6170001698774286e-06, 5...."
solve times,"[0.9185440869998729, 0.7870078619998822, 0.772...","[2.476055438000003, 2.382150292999995, 2.20411...","[3.5875708429998667, 2.9922276750000947, 2.902...","[6.074865407000061, 5.50869654600001, 5.918968...","[11.41466433200003, 10.11940963699999, 9.86841...","[24.889325710999856, 22.752954530999887, 22.79...","[50.62218319699991, 45.52523509499997, 45.4669...","[110.65309568600014, 99.5143704090001, 99.1146...","[238.54708870900026, 200.2781230679998, 200.16...","[627.6542111650001, 435.99840610599995, 433.15..."
integration times,"[0.7635536580000917, 0.7804846480000833, 0.765...","[2.1634452549999423, 2.353078554000149, 2.1765...","[3.1211990610001976, 2.9562169959999665, 2.867...","[5.310349350000024, 5.431525700000066, 5.83711...","[9.894420504999971, 9.952002890999893, 9.69566...","[22.23317843399991, 22.355508762, 22.393126110...","[44.48063865699987, 44.43623002300001, 44.3596...","[95.61041505099979, 96.45851705099994, 96.0667...","[193.30833422799992, 193.6757032820001, 193.53...","[419.75171681500024, 422.16283260399996, 419.6..."
setup time,0.261199,0.457025,0.592527,0.985622,1.884182,3.28134,8.394406,23.740155,81.095357,266.487544
average solve time,0.825927,2.354106,3.160686,5.834177,10.467497,23.478044,47.204799,103.094033,212.997179,498.936466
average integration time,0.769885,2.231028,2.981685,5.526331,9.847363,22.327271,44.425498,96.045239,193.507595,420.533629


# Plot times

Here we define a function to create a plot of setup time, average solve time and average integration time vs number of coupled DFN models, and a loglog plot of integration time vs number of states.

In [3]:
def plot_times(filename, title=None):
    "Plot times for a given filename"
    
    # unpickle times and make lists
    with open("data/" + filename, 'rb') as handle:
        times_and_states = pickle.load(handle)

    npts = list(times_and_states.keys())
    states = [times_and_states[i]["states"] for i in npts]
    setup_times = [times_and_states[i]["setup time"] for i in npts]
    solve_times = [times_and_states[i]["average solve time"] for i in npts]
    integration_times = [times_and_states[i]["average integration time"] for i in npts]

    # create figure
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    title = title or filename[:-4]
    
    # add times to plot
    ax[0].plot(npts, setup_times, "-o", label="Setup time")
    ax[0].plot(npts, solve_times, "-o", label="Solve time")
    ax[0].plot(npts, integration_times, "-o", label="Integration time")
    ax[1].loglog(states, integration_times, "-o")

    # add fit to states vs integration times 
    m, c = np.polyfit(np.log10(states), np.log10(integration_times), 1)
    ax[1].loglog(states, 10**c * states**m, "-")
    ax[1].text(1e3, 10, f"slope = {np.round(m,2)}")

    # add labels
    ax[0].set_xlabel("Number of coupled DFN models")
    ax[0].set_ylabel("Time [s]")            
    ax[0].legend()
    ax[1].set_xlabel("Number of states")
    ax[1].set_ylabel("Integration time [s]")        

    plt.suptitle(title)
    plt.tight_layout()
        

## Plot equipotential pair times 

In [4]:
plot_times("equipotential_pair.pkl")

## Plot isothermal equipotential pair times 

In [5]:
plot_times("isothermal_equipotential_pair.pkl")

## Plot potential pair times 

In [6]:
plot_times("potential_pair.pkl")

## Plot isothermal potential pair times 

In [7]:
plot_times("isothermal_potential_pair.pkl")

# Compare times for equipotential pair and potential pair model

Here we define a function to create a plot to compare setup times, average solve times and average integration times from one or more files.

In [8]:
def compare_times(filenames, labels=None):
    """Compare solve times for different files"""
    
    # create figure
    fig, ax = plt.subplots(2, 2, figsize=(8, 8))
    labels = labels or [filename[:-4] for filename in filenames]
    colors = iter(cm.tab10(np.linspace(0, 1, 10)))
    n = 0
    
    for filename, label in zip(filenames, labels):
        color = next(colors)
        n = n+1
        
        # unpickle times and make lists
        with open("data/" + filename, 'rb') as handle:
            times_and_states = pickle.load(handle)
            
        npts = list(times_and_states.keys())
        states = [times_and_states[i]["states"] for i in npts]
        setup_times = [times_and_states[i]["setup time"] for i in npts]
        solve_times = [times_and_states[i]["average solve time"] for i in npts]
        integration_times = [times_and_states[i]["average integration time"] for i in npts]

        # add times to plot
        ax[0, 0].plot(npts, setup_times, "-o", c=color)
        ax[0, 1].plot(npts, solve_times, "-o", c=color)
        ax[1, 0].plot(npts, integration_times, "-o", c=color)
        ax[1, 1].loglog(states, integration_times, "-o", label=label, c=color)

    # add labels
    ax[0, 0].set_xlabel("Number of coupled DFN models")
    ax[0, 0].set_ylabel("Setup time [s]")   
    ax[0, 1].set_xlabel("Number of coupled DFN models")
    ax[0, 1].set_ylabel("Solve time [s]")        
    ax[1, 0].set_xlabel("Number of coupled DFN models")
    ax[1, 0].set_ylabel("Integration time [s]")       
    ax[1, 1].set_xlabel("Number of states")
    ax[1, 1].set_ylabel("Integration time [s]")        
    if len(filenames) > 1:
        ax[1, 1].legend(loc="lower right")

    plt.tight_layout()
    
    return fig, ax

In [9]:
filenames = ["equipotential_pair.pkl", "potential_pair.pkl"]
_, ax = compare_times(filenames)

In [10]:
filenames = ["potential_pair.pkl", "isothermal_potential_pair.pkl"]
_, ax = compare_times(filenames)